In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits,load_iris
from sklearn.datasets import fetch_openml
from utils import *
from collections import defaultdict
import time
from concurrent.futures import ThreadPoolExecutor
import functools

2023-05-03 11:58:51.703139: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 11:58:52.869071: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-03 11:58:52.869150: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-03 11:58:56.000896: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

Instructions for updating:
non-resource variables are not supported in the long term


Using backend: pytorch


In [3]:
# percs = [0.4, 0.5, 0.6]
# for perc in percs:
#     mnist = fetch_openml('mnist_784')
#     # Split the data into features (X) and labels (y)
#     X = mnist.data.astype('float64')
#     y = mnist.target
#     df = pd.DataFrame(X) 
#     new_df = missing(df,Type='monotone_img',perc=perc,perc_del=0.5, im_width=28,im_height=28)
#     new_df = apply_normalize(new_df)
#     df['label'] = y 
#     new_df['label'] = y
#     corr_gt = np.corrcoef(df.drop('label',axis=1).T)
#     new_df.to_csv(f'DATA/mnist_ms_nm_{perc}.csv',index=False)
#     np.save(f'DATA/corr_gt_{perc}.npy', corr_gt)

In [ ]:
# new_df = new_df.rename(columns={f"pixel{i}": i for i in range(1, 785)})

In [2]:
def rmse_ignore_nan(y_true, y_pred):
    """
    Calculates the Root Mean Squared Error (RMSE) between two matrices, ignoring NaN values.

    Parameters:
    y_true (numpy.ndarray): The true values.
    y_pred (numpy.ndarray): The predicted values.

    Returns:
    float: The RMSE between the true and predicted values, ignoring NaN values.
    """
    # Create a mask of valid elements
    y_true_clean = np.nan_to_num(y_true, nan=-1)
    y_pred_clean = np.nan_to_num(y_pred, nan=-1)
    mask = ~(np.isnan(y_true_clean) | np.isnan(y_pred_clean))

    # Extract only valid elements
    y_true_valid = y_true_clean[mask]
    y_pred_valid = y_pred_clean[mask]
    
    # Calculate RMSE on valid elements
    rmse = np.sqrt(mean_squared_error(y_true_valid, y_pred_valid))
    return rmse

In [6]:
for perc in percs:
    new_df = pd.read_csv(f'DATA/mnist_ms_nm_{perc}.csv')
    corr_dpers = corr_with_dpers(new_df)
    corr_dpers = corr_dpers.drop('label',axis=0)
    corr_dpers = corr_dpers.drop('label',axis=1)
    np.save(f'experiments/Corr/corr_dpers_{perc}.npy', corr_dpers)
    np.save(f'experiments/Rmse/rmse_dpers_{perc}.npy',rmse_ignore_nan(corr_gt,corr_dpers))

100%|██████████| 307720/307720 [23:04<00:00, 222.25it/s]


In [ ]:
perc = 0.4
corr_gt = np.load(f'DATA/corr_gt_{perc}.npy') # REMEMBER that : No Label

In [ ]:
new_df = pd.read_csv(f'DATA/mnist_ms_nm_{perc}.csv') # Have Label , must remove
new_df

In [9]:
from tqdm import tqdm
percs = [0.4, 0.5, 0.6]
for perc in tqdm(percs):
    corr_gt = np.load(f'DATA/corr_gt_{perc}.npy')
    new_df = pd.read_csv(f'DATA/mnist_ms_nm_{perc}.csv')
    corr_mice = np.corrcoef(impute_with_mice(new_df.drop('label',axis=1)).T)
    np.save(f'experiments/Corr/corr_mice_{perc}.npy', corr_mice)
    np.save(f'experiments/Rmse/rmse_mice_{perc}.npy',rmse_ignore_nan(corr_gt,corr_mice))

100%|██████████| 3/3 [59:52<00:00, 1197.34s/it]


In [10]:
from tqdm import tqdm
percs = [0.4, 0.5, 0.6]
for perc in tqdm(percs):
    corr_gt = np.load(f'DATA/corr_gt_{perc}.npy')
    new_df = pd.read_csv(f'DATA/mnist_ms_nm_{perc}.csv')
    corr_missforest = np.corrcoef(impute_with_missforest(new_df.drop('label',axis=1)).T)
    np.save(f'experiments/Corr/corr_missforest_{perc}.npy', corr_missforest)
    np.save(f'experiments/Rmse/rmse_missforest_{perc}.npy',rmse_ignore_nan(corr_gt,corr_missforest))

  0%|          | 0/3 [00:00<?, ?it/s]

Iteration: 0


 33%|███▎      | 1/3 [13:35<27:10, 815.11s/it]

Iteration: 0


 67%|██████▋   | 2/3 [38:25<20:12, 1212.22s/it]

Iteration: 0


100%|██████████| 3/3 [1:12:51<00:00, 1457.16s/it]


In [11]:
from tqdm import tqdm
percs = [0.4, 0.5, 0.6]
for perc in tqdm(percs):
    corr_gt = np.load(f'DATA/corr_gt_{perc}.npy')
    new_df = pd.read_csv(f'DATA/mnist_ms_nm_{perc}.csv')
    corr_softimpute = np.corrcoef(soft_impute_als_df(new_df.drop('label',axis=1)).T)
    np.save(f'experiments/Corr/corr_softimpute_{perc}.npy', corr_softimpute)
    np.save(f'experiments/Rmse/rmse_softimpute_{perc}.npy',rmse_ignore_nan(corr_gt,corr_softimpute))

  0%|          | 0/3 [00:00<?, ?it/s]

[SoftImpute] Max Singular Value of X_init = 1592.278784
[SoftImpute] Iter 1: observed MAE=0.044478 rank=708
[SoftImpute] Stopped after iteration 1 for lambda=31.845576


 33%|███▎      | 1/3 [00:20<00:40, 20.30s/it]

[SoftImpute] Max Singular Value of X_init = 1517.419023
[SoftImpute] Iter 1: observed MAE=0.042604 rank=707
[SoftImpute] Stopped after iteration 1 for lambda=30.348380


 67%|██████▋   | 2/3 [00:39<00:19, 19.90s/it]

[SoftImpute] Max Singular Value of X_init = 1471.822236
[SoftImpute] Iter 1: observed MAE=0.041458 rank=708
[SoftImpute] Stopped after iteration 1 for lambda=29.436445


100%|██████████| 3/3 [01:00<00:00, 20.26s/it]


In [12]:
from tqdm import tqdm
percs = [0.4, 0.5, 0.6]
for perc in tqdm(percs):
    corr_gt = np.load(f'DATA/corr_gt_{perc}.npy')
    new_df = pd.read_csv(f'DATA/mnist_ms_nm_{perc}.csv')
    corr_pca = np.corrcoef(impute_with_pca(new_df.drop('label',axis=1)).T)
    np.save(f'experiments/Corr/corr_pca_{perc}.npy', corr_pca)
    np.save(f'experiments/Rmse/rmse_pca_{perc}.npy',rmse_ignore_nan(corr_gt,corr_pca))

  0%|          | 0/3 [00:00<?, ?it/s]

Stopped after criterion < threshold
Stopped after criterion < threshold
Stopped after criterion < threshold
Stopped after criterion < threshold
Stopped after criterion < threshold
Stopped after 1 iterations


 33%|███▎      | 1/3 [14:35<29:11, 875.74s/it]

Stopped after criterion < threshold
Stopped after criterion < threshold
Stopped after criterion < threshold
Stopped after criterion < threshold
Stopped after criterion < threshold
Stopped after 1 iterations


 67%|██████▋   | 2/3 [31:38<16:01, 961.96s/it]

Stopped after criterion < threshold
Stopped after criterion < threshold
Stopped after criterion < threshold
Stopped after criterion < threshold
Stopped after criterion < threshold
Stopped after 1 iterations


100%|██████████| 3/3 [50:39<00:00, 1013.10s/it]


In [ ]:
from tqdm import tqdm
percs = [0.4, 0.5, 0.6]
for perc in tqdm(percs):
    corr_gt = np.load(f'DATA/corr_gt_{perc}.npy')
    new_df = pd.read_csv(f'DATA/mnist_ms_nm_{perc}.csv')
    corr_knn = np.corrcoef(impute_with_knn(new_df.drop('label',axis=1)).T)
    np.save(f'experiments/Corr/corr_knn_{perc}.npy', corr_knn)
    np.save(f'experiments/Rmse/rmse_knn_{perc}.npy',rmse_ignore_nan(corr_gt,corr_knn))

In [2]:
mnist = fetch_openml('mnist_784')
from tqdm import tqdm
percs = [0.4]
for perc in tqdm(percs):
    X = mnist.data.astype('float32')
    y = mnist.target.astype('int')
    df = pd.DataFrame(X) 
    # try :
    #     df = df.drop('label',axis=1)
    # except:
    #     pass
    df['label'] = y
    df = df.sample(100)
    corr_gt = np.load(f'DATA/corr_gt_{perc}.npy')
    # new_df = pd.read_csv(f'DATA/mnist_ms_nm_{perc}.csv')
    imputed_ginn = ginn_imputate(apply_normalize(df),Type='monotone_img',perc=perc)
    corr_ginn = np.corrcoef(imputed_ginn.drop('label',axis=1).T)
    # np.save(f'experiments/Corr/corr_ginn_{perc}.npy', corr_ginn)
    # np.save(f'experiments/Rmse/rmse_ginn_{perc}.npy',rmse_ignore_nan(corr_gt,corr_ginn))
    print(rmse_ignore_nan(corr_gt,corr_ginn))

  0%|          | 0/1 [00:00<?, ?it/s]

GINN is running on cpu


100%|██████████| 2/2 [00:00<00:00,  4.85it/s]


In [3]:
from tqdm import tqdm # 306p 21s
percs = [0.4, 0.5, 0.6]
for perc in tqdm(percs):
    corr_gt = np.load(f'DATA/corr_gt_{perc}.npy')
    new_df = pd.read_csv(f'DATA/mnist_ms_nm_{perc}.csv')
    corr_EM = np.corrcoef(impute_with_EM(new_df.drop('label',axis=1), n_iter=1).T)
    np.save(f'experiments/Corr/MNIST/EM/corr_{perc}.npy', corr_EM)
    np.save(f'experiments/Rmse/MNIST/EM/rmse_{perc}.npy',rmse_ignore_nan(corr_gt,corr_EM))

100%|██████████| 3/3 [5:06:21<00:00, 6127.21s/it]  


In [2]:
# from tqdm import tqdm # 13p 51s
# percs = [0.4, 0.5, 0.6]
# for perc in tqdm(percs):
#     corr_gt = np.load(f'DATA/corr_gt_{perc}.npy')
#     new_df = pd.read_csv(f'DATA/mnist_ms_nm_{perc}.csv')
#     corr_GAIN = np.corrcoef(gain_impute(new_df.drop('label',axis=1),epochs=10).T)
#     np.save(f'experiments/Corr/MNIST/GAIN/corr_{perc}_v2.npy', corr_GAIN)
#     np.save(f'experiments/Rmse/MNIST/GAIN/rmse_{perc}_v2.npy',rmse_ignore_nan(corr_gt,corr_GAIN))


from tqdm import tqdm # 13p 51s
percs = [0.4, 0.5, 0.6]
path = '/home/pn7hao/Desktop/SecondYear/Research/DeepLearning/'
for perc in tqdm(percs):
    corr_gt = np.load(path + f'DATA/corr_gt_{perc}.npy')
    new_df = pd.read_csv(path + f'DATA/mnist_ms_nm_{perc}.csv')
    corr_GAIN = np.corrcoef(gain_imputed_v2(new_df.drop('label',axis=1),
                                iterations=100,hint_rate=0.9,batch_size=64,alpha=0.5).T)
    np.save(path + f'experiments/Corr/MNIST/GAIN/corr_{perc}.npy', corr_GAIN)
    np.save(path + f'experiments/Rmse/MNIST/GAIN/rmse_{perc}.npy',rmse_ignore_nan(corr_gt,corr_GAIN))

  0%|          | 0/3 [00:00<?, ?it/s]2023-04-29 10:49:36.268666: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-29 10:49:36.268742: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-29 10:49:36.268804: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (hp): /proc/driver/nvidia/version does not exist
2023-04-29 10:49:36.312359: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1/1 [==============================] - 0s 27ms/step


In [4]:
from tqdm import tqdm # 51s
percs = [0.4, 0.5, 0.6]
for perc in tqdm(percs):
    corr_gt = np.load(f'DATA/corr_gt_{perc}.npy')
    new_df = pd.read_csv(f'DATA/mnist_ms_nm_{perc}.csv')
    corr_MeanImpute = np.corrcoef(impute_with_meanImpute(new_df.drop('label',axis=1)).T)
    np.save(f'experiments/Corr/MNIST/MeanImpute/corr_{perc}.npy', corr_MeanImpute)
    np.save(f'experiments/Rmse/MNIST/MeanImpute/rmse_{perc}.npy',rmse_ignore_nan(corr_gt,corr_MeanImpute))

100%|██████████| 3/3 [00:50<00:00, 16.99s/it]


In [2]:
from tqdm import tqdm # 51s
from impyute.imputation.cs import fast_knn
percs = [0.4]
for perc in tqdm(percs):
    corr_gt = np.load(f'DATA/corr_gt_{perc}.npy')
    new_df = pd.read_csv(f'DATA/mnist_ms_nm_{perc}.csv')
    corr_MeanImpute = np.corrcoef(fast_knn(new_df.drop('label',axis=1),k=10).T)
    # np.save(f'experiments/Corr/MNIST/MeanImpute/corr_{perc}.npy', corr_MeanImpute)
    # np.save(f'experiments/Rmse/MNIST/MeanImpute/rmse_{perc}.npy',rmse_ignore_nan(corr_gt,corr_MeanImpute))
    print(corr_MeanImpute)
    print(rmse_ignore_nan(corr_gt,corr_MeanImpute))

  0%|          | 0/1 [00:00<?, ?it/s]